In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from thefuzz import fuzz
from thefuzz import process

cars = pd.read_csv("./concatenated_data.csv", index_col=0)
del cars["description"]
cars.columns = [
    "brand",
    "model",
    "price",
    "mileage",
    "year",
    "location",
    "date_added",
    "body_type",
    "fuel",
    "power",
    "transmission",
    "color",
    "price_new",
    "engine_size",
]
cars.info()


# Overview


In [ ]:
cars.describe()

In [ ]:
def summary_statistics(dataset):
    print(" summary statistics")
    mean = np.round(np.mean(dataset), 2)
    median = np.round(np.nanmedian(dataset), 2)
    min_value = np.round(dataset.min(), 2)
    max_value = np.round(dataset.max(), 2)
    q1 = np.round(dataset.quantile(0.25), 2)
    q3 = np.round(dataset.quantile(0.75), 2)
    # Interquartile range
    iqr = np.round(q3 - q1, 2)
    print("Min: %s" % min_value)
    print("Mean: %s" % mean)
    print("Max: %s" % max_value)
    print("25th percentile: %s" % q1)
    print("Median: %s" % median)
    print("75th percentile: %s" % q3)
    print("Interquartile range (IQR): %s" % iqr)


# Strings


Applying str.title() to each column and specifying the field datatype


In [ ]:
for col in cars.columns:
    if cars[col].dtype == "object":
        cars[col] = cars[col].str.title()


In [ ]:
def explore_strings(df, column):
    # Specifying the field datatype
    df[column] = df[column].fillna("").astype(str)
    strings = df[column].unique()
    print(f"distinct {column}s : ", strings)
    print(f"number of distinct {column}s : ", len(strings) - 1)  # do not count ""
    return strings


## Location


Exploring the values of "location" column


In [ ]:
locations = explore_strings(cars, "location")


In [ ]:
def get_matches(df, column, string_to_match, min_ratio=50):
    strings = df[column].unique()

    # get the top 10 closest matches to our input string
    matches = process.extract(
        string_to_match, strings, limit=10, scorer=fuzz.token_sort_ratio
    )
    # only get matches with a ratio >= min_ratio
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    return close_matches


In [ ]:
def explore_matches(df, column, ratio=60):
    strings = df[column].unique()
    seen = set()
    for s in strings:
        if s in seen:
            continue
        if s == "":
            continue
        close_matches = get_matches(df, column, s, ratio)
        seen.update(close_matches)
        if len(close_matches) >= 2:
            print(f"close matches of {s}: {close_matches}")


In [ ]:
explore_matches(cars, "location")


Unify and fix the values of the "location" column


In [ ]:
my_dict = {"La Manouba": "Manouba", "Gabès": "Gabes"}
cars.replace({"location": my_dict}, inplace=True)


In [ ]:
locations = explore_strings(cars, "location")


## Color


In [ ]:
colors = explore_strings(cars, "color")


Brouillon


In [ ]:
# get all the words that match "gris"
matches = process.extract("gris", colors, limit=20, scorer=fuzz.token_sort_ratio)
# take a look at them to choose min_ratio
matches


In [ ]:
colors_gris = ["gris" if c.lower().startswith("gris") else c for c in colors]

colors_gris = [c[:-4] if c.lower().endswith("gris") else c for c in colors_gris]

matches = process.extract("Gris a", colors, limit=30, scorer=fuzz.token_sort_ratio)
score = fuzz.token_sort_ratio("gris ", "Gris Anthracite")
print(score)
matches

In [ ]:
def replace_matches_in_column(df, column, string_to_match, min_ratio=47):
    # only get matches with a ratio > 90
    close_matches = get_matches(df, column, string_to_match, min_ratio)

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches
    df.loc[rows_with_matches, column] = string_to_match

    # let us know the function's done
    print("All done!")


In [ ]:
def replace_colors(df, column, color):
    shades = [c for c in column if c.lower().strip().startswith(color)]
    rows_with_matches = df["color"].isin(shades)
    df.loc[rows_with_matches, "color"] = color.title()


In [ ]:
replace_colors(cars, colors, "gris")
replace_colors(cars, colors, "bleu")
replace_colors(cars, colors, "blanc")
replace_colors(cars, colors, "vert")
replace_colors(cars, colors, "rouge")
replace_colors(cars, colors, "noir")

Unify and fix the values of the "color" column


In [ ]:
my_dict = {
    "Argent": "Gris",
    "Feuille D'Argent": "Gris",
    "Titanium": "Gris",
    "Miel": "Doré",
    "Champagne": "Jaune",
    "Grenadine": "Rouge",
    "Rouge Bordeaux": "Rouge",
    "Mauve Gris": "Mauve",
    "Aubergine": "Violet",
    "Maran": "Marron",
    "Corail": "Orange",
}
cars.replace({"color": my_dict}, inplace=True)

If color is "**\***" -> color = nan


In [ ]:
specified_color = cars["color"] == "*****"
cars[specified_color] = np.nan


In [ ]:
colors_clean = explore_strings(cars, "color")


## Model


In [ ]:
models = explore_strings(cars, "model")


In [ ]:
already = set()
k = 0
for m in models:
    if m == "":
        continue
    if m in already:
        continue

    close_matches = get_matches(cars, "model", m, 80)
    already.update(set(close_matches))
    if len(close_matches) >= 2:
        k += 1
        print(f"close matches of {m}: {close_matches}")


print(k)


Unify and fix the values of the "model" column


In [ ]:
my_dict = {
    "D-Max": "D Max",
    "H-1": "H 1",
    "Over.Range Rover Sport": "Range Rover Sport",
    "Hr-V": "Hr V",
    "Bt-50": "Bt 50",
    "Hi.Lux": "Hilux",
    "Cx 5": "Cx 5",
    "C-Elysee": "C Elysée",
    "C Elysee": "C Elysée",
    "Cx-9": "Cx 9",
    "Autres": "Autre",
    "X-Trail": "X Trail",
    "Country Man": "Countryman",
    ".Country Man": "Countryman",
    "PickUp": "Pickup",
    "Cee'D": "Ceed",
    "C'Eed": "Ceed",
    "Serie 1": "Série 1",
    "Hi.Ace": "Hi Ace",
    "Gol": "Golf",
    "Volkswagen.Golf": "Golf",
    "Volkswagen.Polo": "Polo",
}
cars.replace({"model": my_dict}, inplace=True)

In [ ]:
models_clean = explore_strings(cars, "model")


## Brand


In [ ]:
brands = explore_strings(cars, "brand").sort()


In [ ]:
explore_matches(cars, "brand", 70)


In [ ]:
my_dict = {
    "Mercedes-Benz": "Mercedes Benz",
    "Mercedes": "Mercedes Benz",
    "Land-Rover": "Land Rover",
}
cars.replace({"brand": my_dict}, inplace=True)


In [ ]:
brands = explore_strings(cars, "brand")


## Body Type


In [ ]:
body_types = explore_strings(cars, "body_type")


## Fuel


In [ ]:
fuels = explore_strings(cars, "fuel")


Unify and fix the values of the "fuel" column


In [ ]:
my_dict = {
    "Hybride (Essence/Électrique)": "Hybride",
    "Hybride (Diesel/Électrique)": "Hybride",
}
cars.replace({"fuel": my_dict}, inplace=True)

In [ ]:
fuel_clean = explore_strings(cars, "fuel")

## Transmission


In [ ]:
transmissions = explore_strings(cars, "transmission")


Unify and fix the values of the "transmission" column


In [ ]:
my_dict = {
    "Mécanique": "Manuelle",
}
cars.replace({"transmission": my_dict}, inplace=True)


In [ ]:
transmission_clean = explore_strings(cars, "transmission")


Convert the column "engine_size" to float


In [ ]:
contains_symbol = cars["engine_size"].str.contains("[><]") & cars["engine_size"].notna()
cars.loc[contains_symbol, "engine_size"] = cars.loc[
    contains_symbol, "engine_size"
].apply(lambda string: string[1:])
cars.loc[cars["engine_size"].notna(), "engine_size"] = cars.loc[
    cars["engine_size"].notna(), "engine_size"
].apply(lambda string: string[:-1])
cars["engine_size"] = cars["engine_size"].astype(np.float64)


'' to nan

In [ ]:
cars = cars.replace('', np.nan)
# cars.info()

# print( cars["color"].unique())

# Dropping duplicates


In [ ]:
cars.drop_duplicates(inplace=True)


In [ ]:
cars.to_csv("./new_clean_data.csv")
cars.info()


# Dates


Turn "date_added" column to datetime


In [ ]:
contains_hyphen = cars["year"].str.contains("-") & cars["year"].notna()
cars.loc[contains_hyphen, "year"] = cars.loc[contains_hyphen, "year"].apply(
    lambda string: string.split("-")[1]
)
contains_point = cars["year"].str.contains("[.]") & cars["year"].notna()
cars.loc[contains_point, "year"] = cars.loc[contains_point, "year"].apply(
    lambda string: string.split(".")[-1]
)
cars["year"] = cars["year"].astype(np.float64)


If 10<= year <= 23 -> year += 2000, if 60 <= year -> year += 1900, else year = nan


In [ ]:
year_between_0_and_23 = (cars["year"] >= 10) & (cars["year"] <= 23)
cars.loc[year_between_0_and_23, "year"] += 2000
year_more_than_60 = (cars["year"] >= 60) & (cars["year"] <= 99)
cars.loc[year_more_than_60, "year"] += 1900
correct_year = (cars["year"] >= 1960) & (cars["year"] <= 2023)
cars.loc[~correct_year, "year"] = np.nan


Remove month from year in the "year" column and convert it to float


In [ ]:
cars["date_added"] = pd.to_datetime(cars["date_added"])


EDA

In [ ]:
# year_between_0_and_23 = (cars["year"] >= 10) & (cars["year"] <= 23)
# cars.loc[year_between_0_and_23, "year"] += 2000
years=cars["year"].sort_values().unique()
years

# Numerical Data


## Mileage


mileage -> abs(mileage)


In [ ]:
cars["mileage"] = abs(cars["mileage"])


If mileage < 10 -> mileage = nan


In [ ]:
mileage_less_than_10 = cars["mileage"] < 10
cars.loc[mileage_less_than_10, "mileage"] = np.nan


If mileage <= 1000 -> mileage \*= 1000


In [ ]:
mileage_less_than_1000 = cars["mileage"] <= 1000
cars.loc[mileage_less_than_1000, "mileage"] *= 1000


If mileage >= 1 000 000 -> mileage = nan


In [ ]:
mileage_over_1000000 = cars["mileage"] >= 1000000
cars.loc[mileage_over_1000000, "mileage"] = np.nan


In [ ]:
mileage_over_1000000 = cars["mileage"] <5000
cars.loc[mileage_over_1000000, ["model","price","mileage", "year"]].sort_values(by="mileage", ascending=False)

# conclusion : milegae < 4000 & year <= 2020 -> np.nan 

In [ ]:
mileage_under_4000 = cars["mileage"] <4000
old=cars["year"]< 2020
cars.loc[mileage_under_4000 & old, ["model","price","mileage", "year"]].sort_values(by="mileage", ascending=False)


In [ ]:
cars.loc[mileage_under_4000 & old, "mileage"]=np.nan


## Getting outliers


In [ ]:
def get_outliers_z_score(df, column, threshold=4):
    mean = np.mean(df[column])
    std = np.std(df[column])
    return abs((df[column] - mean) / std) > threshold


In [ ]:
def get_outliers_iqr(df, column):
    q1 = np.percentile(df.loc[df[column].notna(), column], 25)
    q3 = np.percentile(df.loc[df[column].notna(), column], 75)

    iqr = q3 - q1

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return (df[column] > upper_bound) | (df[column] < lower_bound)


## Power


Convert N.D to nan in the "power" column


In [ ]:
power_is_ND = cars["power"] == "N.D"
cars.loc[power_is_ND, "power"] = np.nan


Convert the column "power" to float


In [ ]:
contains_CV = cars["power"].str.contains("C") & cars["power"].notna()
cars.loc[contains_CV, "power"] = cars.loc[contains_CV, "power"].apply(
    lambda string: string.split()[0]
)
cars["power"] = cars["power"].astype(np.float64)


If power > 200 -> power = nan


In [ ]:
power_over_200 = cars["power"] > 200
cars.loc[power_over_200, "power"] = np.nan

Get Mode Power per model and change the value of power if it exceeds the iqr range per model


In [ ]:
model_to_power_mode = (
    cars.groupby("model")["power"]
    .apply(lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan)
    .to_dict()
)
model_to_upper_bound = {}
for model in cars["model"].unique():
    try:
        df = cars.groupby("model").get_group(model)
    except KeyError:
        continue

    if df.empty:
        continue

    power_values = df.loc[df["power"].notna(), "power"]

    if power_values.empty:
        continue

    q1 = np.percentile(power_values, 25)
    q3 = np.percentile(power_values, 75)

    iqr = q3 - q1

    upper_bound = q3 + 1.5 * iqr
    model_to_upper_bound[model] = upper_bound

for model in cars["model"].unique():
    df = cars[cars["model"] == model]

    power_values = df.loc[df["power"].notna(), "power"]

    if power_values.empty:
        continue

    cars.loc[(cars["model"] == model) & (cars["power"] >= model_to_upper_bound[model]),
        "power"
    ] = model_to_power_mode[model]

## Price


In [ ]:
summary_statistics(cars["price"])
cars["price"].hist(bins=4)
plt.xscale("log")
plt.show()


price -> abs(price)
If price is 0 -> price = nan


In [ ]:
cars["price"] = abs(cars["price"])
price_is_0 = cars["price"] <= 0
cars.loc[price_is_0, "price"] = np.nan


If price over 1_000_000 -> price = nan


In [ ]:
price_over_1_000_000 = cars["price"] >= 1e6
cars.loc[price_over_1_000_000, "price"] = np.nan



If price less than 500 -> price \*= 1000


In [ ]:
print(
    cars.loc[
        (cars["model"] =="206") & (cars["price"] < 100000),
        ["brand", "model", "year", "mileage", "price"],
    ].sort_values(by=["price", "year"], ascending=[False, True])
)
cars.loc[(cars["price"] > 1) & (cars["price"] < 500), ["price", "year"]].hist(bins=5)
plt.show()


In [ ]:
price_less_than_1000 = cars["price"] <= 500
cars.loc[price_less_than_1000, "price"] *= 1000

In [ ]:
ax = sns.boxplot(
    data=cars[["price"]],
    width=0.5,
    orient="h",
    flierprops=dict(markeredgecolor="#D94D1F"),
)
plt.xscale("log")
plt.show()


price < 4000 -> np.nan

In [ ]:
uncoherent_pattern=(cars["price"] ==1234) | (cars["price"] ==1111) | (cars["price"] <=1000)
data=cars.loc[uncoherent_pattern,
        ["brand", "model", "year", "mileage", "price"],
    ].sort_values(by=["price","year"], ascending=[True, True])
print(data)
cars.loc[uncoherent_pattern,
        "price",
    ]=np.nan

In [ ]:

under_4000=(cars["price"] <=4000)

data=cars.loc[under_4000,
        ["brand", "model", "year", "mileage", "price"],
    ].sort_values(by=["year","price"], ascending=[False, True])
print(data)
plt.hist(data["year"], bins=7, alpha=0.5, label=model)




In [ ]:
recent=cars["year"]>2000
cars.loc[under_4000 & recent,
        ["price"],
    ]=np.nan

price grouped by model

In [ ]:
def calIQR(x):
    q3=x.quantile(.75)
    q1=x.quantile(.25)
    iqr=q3-q1
    lower_bound=q1-1.5*iqr
    upper_bound=q3+1.5*iqr
    return (lower_bound,upper_bound)

def callZscore(x):
    mean = np.mean(x, axis=0)
    std = np.std(x,axis=0)
    return abs((x - mean) / std)
    # return zscore(x)

    




IQR

In [ ]:
cars_grouped=cars.groupby("model")
bounds = cars_grouped["price"].apply(calIQR)
# print(my_dict)

In [ ]:

def histogram_by_model(grouped,model,column):
    grouped=cars.groupby("model")
    rows=grouped.get_group(model)
    summary_statistics(rows[column])

    plt.hist(rows[column], bins=7, alpha=0.5, label=model)

def boxplot_by_model(grouped,model,column):
    grouped=cars.groupby("model")

    rows=grouped.get_group(model)
    sns.boxplot(
    data=rows[[column]], orient="h", flierprops=dict(markeredgecolor="#D94D1F")
)

    plt.show()



In [ ]:
model="206"
histogram_by_model(cars_grouped,model,"price")
# boxplot_by_model(cars_grouped,model,"price")

In [ ]:

models=cars['model'].unique()
lower=bounds[model][0]
upper=bounds[model][1]

# print(lower,upper)

cars.loc[(cars["model"] == model) & ((cars["price"] < lower) | (cars["price"] > upper )),
            "price"
        ] = np.nan


sum=0
for model in models:
    if  model not in bounds.keys()  : 
        continue

    if str(bounds[model][0]) == 'nan':
        continue
    lower=bounds[model][0]
    upper=bounds[model][1]
   
    print(lower,upper)
    out=cars.loc[(cars["model"] == model) & ((cars["price"] < lower) | (cars["price"] > upper )),
            "price"
        ] 
    sum+=out.shape[0]
    out=np.nan
    
        
print(sum)
# boxplot_by_model(cars_grouped,"Rio","price")
# histogram_by_model(cars_grouped,model,"price")



In [ ]:

data=cars.loc[(cars["price"] <4000) ,
        ["brand", "model", "year", "mileage", "price"],
    ].sort_values(by=["price", "year"], ascending=[False, True])
print(data
    
)


## z_score


In [ ]:
models=cars["model"].unique()
threshold=2
for model in models:
    if str(model) =='nan' : 
        continue 
    df=cars_grouped.get_group(model)
    df["z_score"]=callZscore(df["price"])
    above_threshold=df["z_score"]> threshold
    outliers=df.loc[above_threshold,["z_score","price","year","mileage","price_new"]]
    indices=outliers.index.tolist()
    cars.loc[indices,"price" ]= np.nan

    




In [ ]:
cars["price"].hist(bins=20)
plt.yscale("log")
plt.show()
summary_statistics(cars["price"])


In [ ]:
category1=cars.loc[cars["price"]<100_000,"price"]
category2=cars.loc[cars["price"]>=100_000,"price"]
category1.hist(bins=10)
plt.yscale("log")
plt.show()
summary_statistics(category1)


In [ ]:
cars.loc[cars["price"] > 400_000,["brand", "model", "year", "price"]]


In [ ]:
sns.boxplot(
    data=cars[["price"]], orient="h", flierprops=dict(markeredgecolor="#D94D1F")
)
plt.xscale("log")

plt.show()
